# concern classifier

In [2]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf
print(tf.__version__)

2.18.0


In [3]:
from transformers import TFBertForSequenceClassification, BertTokenizer
import tensorflow as tf

# Load a pre-trained BERT model for TensorFlow
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=5) # assuming 5 categories

def classify_concern(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="tf")
    # Run the inputs through the model
    outputs = model(inputs)
    # Get the predicted class
    predicted_class = tf.argmax(outputs.logits, axis=1).numpy()[0]
    # Define the categories
    categories = ["Anxiety", "Depression", "Stress", "Insomnia", "Eating Disorder"]
    return categories[predicted_class]

# Example usage
concern = classify_concern("feeling very anxious")
print("Classified Concern:", concern)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

2024-10-27 01:39:02.637057: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Classified Concern: Depression


# polarity finder

# not working

In [4]:
from transformers import pipeline

# Load sentiment analysis pipeline from Hugging Face
sentiment_analyzer = pipeline("sentiment-analysis")

def find_polarity(text):
    result = sentiment_analyzer(text)
    polarity = result[0]['label']
    score = result[0]['score']
    return polarity, score

# Example usage
text = "I feel very anxious lately"
polarity, score = find_polarity(text)
print(f"Polarity: {polarity}, Score: {score}")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Polarity: NEGATIVE, Score: 0.9955092668533325


# keyword extractor 

In [9]:
import spacy

# Load spaCy model (a pre-trained model that could be fine-tuned for specific mental health terms)
nlp = spacy.load("en_core_web_sm")

def extract_keywords(text):
    doc = nlp(text)
    keywords = [ent.text for ent in doc.ents if ent.label_ == "MENTAL_HEALTH"]
    return keywords

# Example usage
text = "I've been feeling very anxious lately."
keywords = extract_keywords(text)
print("Extracted Keywords:", keywords)

Extracted Keywords: []


# intensify scoring

In [10]:
def score_intensity(text):
    if "very" in text or "extremely" in text:
        return 8
    elif "slightly" in text or "a bit" in text:
        return 4
    else:
        return 6  # default intensity

# Example usage
intensity = score_intensity("I feel extremely anxious")
print("Intensity Score:", intensity)

Intensity Score: 8


# time based

In [11]:
from datetime import datetime

# To store results in a timeline
timeline_data = []

def track_timeline(input_text):
    timestamp = datetime.now()
    polarity, score = find_polarity(input_text)
    keywords = extract_keywords(input_text)
    concern_category = classify_concern(" ".join(keywords))
    intensity = score_intensity(" ".join(keywords))
    
    entry = {
        "timestamp": timestamp,
        "polarity": polarity,
        "score": score,
        "keywords": keywords,
        "concern_category": concern_category,
        "intensity": intensity
    }
    timeline_data.append(entry)

    # Analyze sentiment shift
    if len(timeline_data) > 1:
        previous_entry = timeline_data[-2]
        if entry["polarity"] != previous_entry["polarity"]:
            shift = f"Shift from {previous_entry['polarity']} to {entry['polarity']}"
        else:
            shift = "No significant sentiment shift"
    else:
        shift = "First entry, no shift analysis"
    
    return entry, shift

# Example usage over a timeline
track_timeline("I feel very low and can’t sleep well.")
track_timeline("I feel a bit better but still anxious.")
for data in timeline_data:
    print(data)

{'timestamp': datetime.datetime(2024, 10, 27, 1, 48, 30, 855199), 'polarity': 'NEGATIVE', 'score': 0.9996077418327332, 'keywords': [], 'concern_category': 'Depression', 'intensity': 6}
{'timestamp': datetime.datetime(2024, 10, 27, 1, 48, 31, 233277), 'polarity': 'NEGATIVE', 'score': 0.9623287320137024, 'keywords': [], 'concern_category': 'Depression', 'intensity': 6}


# full integration

In [12]:
def process_user_input(input_text):
    polarity, _ = find_polarity(input_text)
    keywords = extract_keywords(input_text)
    concern = classify_concern(" ".join(keywords))
    intensity = score_intensity(" ".join(keywords))
    
    entry, shift = track_timeline(input_text)
    
    return {
        "polarity": polarity,
        "keywords": keywords,
        "concern": concern,
        "intensity": intensity,
        "shift": shift
    }

# Example sequence of inputs
inputs = [
    "I can’t sleep well and I feel very low.",
    "I feel a bit better but still anxious."
]

for input_text in inputs:
    result = process_user_input(input_text)
    print(result)

{'polarity': 'NEGATIVE', 'keywords': [], 'concern': 'Depression', 'intensity': 6, 'shift': 'No significant sentiment shift'}
{'polarity': 'NEGATIVE', 'keywords': [], 'concern': 'Depression', 'intensity': 6, 'shift': 'No significant sentiment shift'}
